In [36]:
import pandas as pd
import os
from PIL import Image
import pytesseract

In [37]:
df = pd.read_csv("peabody_files/Peabody_Extended_Fields_Acc.1-12.csv")

In [49]:
images = []
i = 0
for folder in os.listdir("peabody_files/Accession Files/"):
    if 'DS_Store' in folder:
        continue
    for file in os.listdir("peabody_files/Accession Files/"+folder):
        img = Image.open("peabody_files/Accession Files/"+folder+'/'+file)
#         img.crop((0,0, 35, 150))
        catNumber = img.crop((10,10, 300, 100))
        print(pytesseract.image_to_string(catNumber))

Cat. No. 1
Cat. No. 2
Cat. No. 3
Cat. No. 4
Cat. No. 5
Cat. No. 6
Cat. No. 7
Cat. No. 8
Cat. No. 9
Cat. No. 1C
Cat. No. l]
Cat. No. 12
Cat. No. 12
Cat. No. 14
Cat. No. 15
Cat. No. 1(
Cat. No. 17
Cat. No. 18
Cat. No. 19
Cat. No. 3(
Cat. No. 2]
Cat. No. 2 2
Cat. No. 23
Cat. No. 24
Cat. No. 25
Cat. No. 26
Cat. No. 37
Cat. No. 28
Cat. No. 39
Cat. No. 30
Cat. No. 31
Cat. No. 32
Cat. No. 33
Cat. No. 34
Cat. No. 35
Cat. No. 36
Cat. No. 37
Cat. No. 38
Cat. No. 39
Cat. No. 40
Cat. No. 41
Cat. No. 42
Cat. No. 45
Cat. No. 44
Cat. No. 45
Cat. No. 46
Cat. No. 47
Cat. No. 48
Cat. No. 49
Cat. No. 50
Cat. No. 51
Cat. No. 52
Cat. No. 53
Cat. No. 54
Cat. No. 55
Cat. No. 56
Cat. No. 57
Cat. No. 58
Cat. No. 59
Cat. No. 60
Cat. No. 61
Cat. No. 68
Cat. No. 65
Cat. No. 64
Cat. No. 65
Cat. No. 66


OSError: cannot identify image file 'peabody_files/Accession Files/Acc. No. 16/16_1188.pdf'